In [1]:
# Data Loader - 웹페이지 데이터 가져오기
import json
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

1. 데이터 로드
2. 텍스트 분할
3. 인덱싱(임베딩 후 인덱싱)
4. 검색(retrieval)

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
import PIL
import PIL.Image
print(PIL.Image.MAX_IMAGE_PIXELS)
PIL.Image.MAX_IMAGE_PIXELS = None
print(PIL.Image.MAX_IMAGE_PIXELS)


250000000
None


In [6]:
import ocrmypdf
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

ocrmypdf.ocr(
    "datasets/manual.pdf",
    "datasets/ocr_output.pdf",
    language="kor",
    deskew=True,
    force_ocr=True,
    max_image_mpixels=None
)

Output()

Output()

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

...

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\PIL\Image.py:3442: DecompressionBombWarning: Image size 
(435071142 pixels) exceeds limit of 250000000 pixels, could be decompression bomb DOS attack.
  warnings.warn(

[tesseract] lots of diacritics - possibly poor OCR

DecompressionBombError: Image size (1262328755 pixels) exceeds limit of 500000000 pixels, could be decompression bomb DOS attack.

In [15]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode

pipeline_options = PdfPipelineOptions(
    do_table_structure = True,
    do_text_extraction=True,
)
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

filepath = 'datasets/ocr_output.pdf'
doc = doc_converter.convert(filepath)

markdown_text = doc.document.export_to_markdown()
# print(markdown_text)
#읽지 못하는 텍스트와 사진 처리방법 고민

c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(
c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:267: RuntimeWarning: Mean of empty slice
  np.nanmean(


In [ ]:
# from docling.chunking import HybridChunker

# chunker = HybridChunker(
#     tokenizer='BAAI/bge-m3',
#     max_token_length=1024,
#     overlap=200
# )

# chunks = list(chunker.chunk(doc.document))

In [16]:
splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
) #잘 안돼면 512, 768, 1024 등으로 변경
chunks = splitter.create_documents([markdown_text])

# 3. Document 객체 생성
documents = [Document(page_content=chunk.page_content, metadata={"source": filepath}) for chunk in chunks]

In [17]:
# embedding_model = HuggingFaceEmbeddings(
#     model_name='BAAI/bge-m3',
#     model_kwargs={'device':device},
#     encode_kwargs={'normalize_embeddings':True},
# )

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

retriever = vectorstore.as_retriever()

In [18]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [19]:
prompt = ChatPromptTemplate.from_template("""
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
You must answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:
""")

In [20]:
from langchain_ollama import OllamaLLM

# llm = Ollama(model="llama3.2:3b")
# llm = Ollama(model="deepseek-r1:1.5b")
llm = OllamaLLM(model="gemma3:4b")
# llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

rag_chain = (
    {'context': compression_retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
question = "총 사업기간은 어떻게 돼?"
result = rag_chain.invoke(question)
print(result)

'24. 1. 1. ~ '24. 12. 31. 기간으로 총 사업기간이 정해져 있습니다.


In [ ]:
# #pdf내의 표를 저장함
# from pathlib import Path

# def extract_and_export_tables(input_path, output_dir):
#     output_dir.mkdir(parents=True, exist_ok=True)
    
#     doc_converter = DocumentConverter()
    
#     conv_res = doc_converter.convert(input_path)
#     doc_filename = conv_res.input.file.stem
    
#     for table_ix, table in enumerate(conv_res.document.tables):
#         table_df = table.export_to_dataframe()
        
#         csv_filename = output_dir / f"{doc_filename}-table-{table_ix+1}.csv"
#         table_df.to_csv(csv_filename)
        
#         html_filename = output_dir / f"{doc_filename}-table-{table_ix+1}.html"
#         with html_filename.open("w") as fp:
#             fp.write(table.export_to_html())
            
#     table_count = len(conv_res.document.tables) 
    
#     return table_count


# output_dir = Path("output")
# output_dir.mkdir(parents=True, exist_ok=True)

# table_count = extract_and_export_tables(filepath, output_dir)
# print("success")

c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(
c:\Users\minja\anaconda3\envs\virtual\lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:267: RuntimeWarning: Mean of empty slice
  np.nanmean(
Usage of TableItem.export_to_html() without `doc` argument is deprecated.
Usage of TableItem.export_to_html() without `doc` argument is deprecated.
Usage of TableItem.export_to_html() without `doc` argument is deprecated.


success


In [ ]:
# from langchain_core.prompts import PromptTemplate
# from langchain_docling.loader import ExportType

# filepath = 'datasets/여비산정기준표.pdf'
# embedding_model = 'BAAI/bge-m3'
# reranking_model = 'Alibaba-NLP/gte-multilingual-reranker-base'
# export_type = ExportType.DOC_CHUNKS

# prompt = PromptTemplate.from_template("""
# You are an assistant for question-answering tasks. 
# Use the following pieces of retrieved context to answer the question.
# If the user asks for a simple answer, summarize the key points.
# If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
# You must answer in Korean.

# #Context: 
# {context}

# #Question:
# {question}

# #Answer:
# """)

# from langchain_docling import DoclingLoader
# from docling.chunking import HybridChunker

# loader = DoclingLoader(
#     file_path = filepath,
#     export_type = ExportType.DOC_CHUNKS,
#     chunker = HybridChunker(tokenizer=embedding_model)
# )

# docs = loader.load()

# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_milvus import Milvus

# MUIVUS_URI = "./milvus_example.db"

# embedding = HuggingFaceEmbeddings(
#     model_name=embedding_model,
#     model_kwargs={'device':device,
#                   'trust_remote_code':True}
# )

# vectorstore = Milvus.from_documents(
#     documents=docs,
#     embedding=embedding,
#     collection_name="docling_transformer",
#     connection_args={
#         "url":MUIVUS_URI,
#         "db_name":" edu"
#     },
#     index_params={
#         "index_type":"FLAT",
#         "metric_type": "cosine"
#     },
#     drop_old=True
# )